# Trabalho Prático 01 - Classificação KNN

A aplicação de IA utiliza o algoritmo KNN (K-Neighrest-Neighbors), onde foi testado sua implementação feita a mão, e a oferecida pelo sklearn. A passo a passo ele está detalhado abaixo.

In [ ]:
%pip install scikit-learn pandas numpy

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import math
from collections import Counter

In [ ]:
# Lendo o arquivo CSV das flores.
dados = pd.read_csv('Iris.csv')

# Dividir em treino e teste (ex: 80% treino, 20% teste)
dados_treinamento, dados_teste = train_test_split(dados, test_size=0.2, random_state=42, shuffle=True)

# Resetar index para evitar problemas ao iterrows
dados_treinamento = dados_treinamento.reset_index(drop=True)
dados_teste = dados_teste.reset_index(drop=True)

## Aplicação - Implementação desenvolvida a mão

In [ ]:
def distancia_euclidiana():
    resultado = []
    
    for _, row1 in dados_teste.iterrows():
        distancias = []
        for _,row2 in dados_treinamento.iterrows():
            d = math.sqrt((row1["SepalLengthCm"] - row2["SepalLengthCm"])**2+
                          (row1["SepalWidthCm"] - row2["SepalWidthCm"])**2+
                          (row1["PetalLengthCm"] - row2["PetalLengthCm"])**2+
                          (row1["PetalWidthCm"] - row2["PetalWidthCm"])**2
                          )
            distancias.append((d, row2.Id, row2.Species))
        distancias.sort(key= lambda x: x[0])
        resultado.append(distancias)
    
    return resultado


distancias = distancia_euclidiana()

In [ ]:
quantidade_vizinhos = 7

def knn_prever(k = 3):
    vizinhos = []
    for distancia in distancias:
        vizinhos_aux = []
        for i in range(k):
            vizinhos_aux.append(distancia[i])
        vizinhos.append(vizinhos_aux)
    return vizinhos

vizinhos = knn_prever(quantidade_vizinhos)

In [ ]:
def contar_vizinhos():
    contador_vizinhos = []
    for viz in vizinhos:
        species = [v[2] for v in viz]
        contador_vizinhos.append(Counter(species))
    return contador_vizinhos

contador_vizinhos = contar_vizinhos()
especies_preditas = [c.most_common(1)[0][0] for c in contador_vizinhos]

dados_teste['SpeciesPredictions'] = especies_preditas

In [ ]:
# Obter todas as classes únicas (de y_true e y_pred)
classes = np.unique(dados_teste[["Species", "SpeciesPredictions"]].values)
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Converter y_true e y_pred para índices
y_true_idx = [class_to_index[c] for c in dados_teste.Species]
y_pred_idx = [class_to_index[c] for c in dados_teste.SpeciesPredictions]

In [ ]:
# Matriz de confusão manual
def confusion_matrix_manual(y_true, y_pred, n_classes):
    cm = np.zeros((n_classes, n_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

cm = confusion_matrix_manual(y_true_idx, y_pred_idx, n_classes=len(classes))

In [ ]:
def accuracy_manual(cm):
    correct = np.trace(cm)  # soma da diagonal
    total = cm.sum()
    return correct / total

acc = accuracy_manual(cm)

In [ ]:
def precision_manual(cm):
    precisions = []
    for i in range(len(cm)):
        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        precisions.append(tp / (tp + fp) if (tp + fp) > 0 else 0)
    return np.mean(precisions)

prec = precision_manual(cm)

In [ ]:
def recall_manual(cm):
    recalls = []
    for i in range(len(cm)):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        recalls.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
    return np.mean(recalls)

rec = recall_manual(cm)

In [ ]:
print("Implementação da KNN Manual (Sem uso de biblioteca)")
print(f"===== f = {quantidade_vizinhos} ========")

# Transformando em DataFrame para adicionar nomes
cm_df = pd.DataFrame(cm, index=classes, columns=classes)
print("Matriz de Confusão:\n", cm_df)
print(f"\nAcurácia: {acc:.2f}")
print(f"Precisão: {prec:.2f}")
print(f"Revocação: {rec:.2f}")

## Aplicação - Implementação da biblioteca Sklearn

In [ ]:
# Certificando que só temos as colunas de features
features = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']  # adapte conforme seu CSV

In [ ]:
X_train = dados_treinamento[features]
y_train = dados_treinamento["Species"]

X_test = dados_teste[features]  # SEM colunas extras!
y_test = dados_teste["Species"]

In [ ]:
# Padronizar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  # agora não vai dar erro

In [ ]:
# Treinar KNN
k = 7
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

In [ ]:
# Prever
y_pred = knn.predict(X_test)

In [ ]:
# Avaliar
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))